In [1]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
from tabulate import tabulate
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse

In [3]:
import json
with open('GA_output_ET.json', 'r') as fp:
    feature_list = json.load(fp)

In [4]:
feature_list

{'SYN': ['Src Port',
  'Tot Fwd Pkts',
  'Tot Bwd Pkts',
  'TotLen Bwd Pkts',
  'Fwd Pkt Len Max',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Min',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Std',
  'Flow IAT Mean',
  'Flow IAT Std',
  'Flow IAT Max',
  'Flow IAT Min',
  'Fwd IAT Tot',
  'Fwd IAT Mean',
  'Fwd IAT Max',
  'Fwd IAT Min',
  'Bwd IAT Max',
  'Fwd Pkts/s',
  'Bwd Pkts/s',
  'Pkt Len Max',
  'Pkt Len Std',
  'SYN Flag Cnt',
  'PSH Flag Cnt',
  'Pkt Size Avg',
  'Subflow Fwd Pkts',
  'Subflow Fwd Byts',
  'Subflow Bwd Byts',
  'Fwd Act Data Pkts',
  'Active Std',
  'Active Max',
  'Idle Mean',
  'Idle Std',
  'Idle Max',
  'Idle Min',
  'Label'],
 'HTTP': ['Dst Port',
  'Flow Duration',
  'Flow Pkts/s',
  'Flow IAT Max',
  'Fwd IAT Tot',
  'Fwd IAT Min',
  'Fwd Pkts/s',
  'FIN Flag Cnt',
  'SYN Flag Cnt',
  'Label'],
 'ACK': ['TotLen Bwd Pkts',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Mean',
  'Fwd Pkt Len Std',
  'Flow Byts/s',
  'Flow IAT Std',
  'Fwd IAT Tot',
  'Fwd IAT Mean',
  'Fwd IAT 

In [5]:
file_list={
'SYN': ['../csvs\\dos-synflooding-1-dec.pcap_Flow.csv', '../csvs\\dos-synflooding-6-dec.pcap_Flow.csv'],
'HTTP': ['../csvs\\mirai-httpflooding-4-dec.pcap_Flow.csv', '../csvs\\http-flood -maze.csv'],
'ACK': ['../csvs\\mirai-ackflooding-4-dec.pcap_Flow.csv', '../csvs\\ack_maze.csv'],
'UDP': ['../csvs\\mirai-udpflooding-4-dec.pcap_Flow.csv', '../csvs\\UDP_DDoS_00001.pcap_Flow.csv'],
'ARP': ['../csvs\\mitm-arpspoofing-6-dec.pcap_Flow.csv', '../csvs\\mitm-arpspoofing-5-dec.pcap_Flow.csv'],
'SP': ['../csvs\\scan-hostport-3-dec.pcap_Flow.csv', '../csvs\\Edge-Port Scanning attack.pcap_Flow.csv'],
'BF': ['../csvs\\mirai-hostbruteforce-5-dec.pcap_Flow.csv','../csvs\\edges-password attacks.pcap_Flow.csv'],
'OS': ['../csvs\\scan-portos-3-dec.pcap_Flow.csv', '../csvs\\BoT-IoT_OSScan.csv'],
'SCHD': ['../csvs\\scan-hostport-3-dec.pcap_Flow.csv', '../csvs\\scan-hostport-5-dec.pcap_Flow.csv'],
'MHDis': ['../csvs\\mirai-hostbruteforce-3-dec.pcap_Flow.csv', '../csvs\\mirai-hostbruteforce-4-dec.pcap_Flow.csv']}

In [11]:
param_grid = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty': ['none', 'l1', 'l2', 'elasticnet'],  # We'll filter penalties in the loop based on the solver
    'C': loguniform(1e-5, 100)
}

# Custom function to filter invalid penalty values based on the solver
def get_valid_penalties(solver):
    if solver == 'lbfgs' or solver == 'newton-cg':
        return ['none', 'l2']  # Valid penalties for 'lbfgs' and 'newton-cg'
    elif solver == 'liblinear':
        return ['l1', 'l2', 'elasticnet']  # Valid penalties for 'liblinear'
    else:
        return ['none', 'l2']  # Default for other solvers


In [12]:
def run_random_search(model, param_grid, x_train, y_train):
    # Create parameter grid with valid penalties
    valid_penalties = get_valid_penalties(param_grid['solver'][0])  # Assuming first solver in the list is selected
    param_grid['penalty'] = [pen for pen in param_grid['penalty'] if pen in valid_penalties]

    # Perform random search with cross-validation
    grid = RandomizedSearchCV(model, param_grid, cv=ps, scoring='f1_macro')
    grid.fit(x_train, y_train)
    return grid.best_params_, round(grid.best_score_, 8), grid.best_estimator_

In [7]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

# RandomizedSearchCV  DT

In [8]:
lines=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]

for j in file_list:
    print(j)
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  

    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))

    nfolds=10
    param_grid = { 'criterion':['gini','entropy'],
                      "max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                     "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                        # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                        "max_features" : sp_randint(1,X_train.shape[1])}

    second=time()
    f1=[]
    clf=DecisionTreeClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(50)):
        second=time()
        a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(10):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("DT_HPO.csv",index=False)

final_parametres=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9922977652517754   3.003    24      


100%|██████████| 50/50 [01:33<00:00,  1.87s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.7      24      


100%|██████████| 50/50 [00:18<00:00,  2.63it/s]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.49670078103959076  0.741    24      


...

SCHD
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9876450359316712   0.368    24      


100%|██████████| 50/50 [00:13<00:00,  3.62it/s]


MHDis
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.526428294886419    0.755    24      


100%|██████████| 50/50 [00:30<00:00,  1.65it/s]


    criterion      max_depth    max_features    min_samples_split         F1          Std    Time    No  Attack
--  -----------  -----------  --------------  -------------------  ---------  -----------  ------  ----  --------
 0  gini                  24              20                    8  0.994223   3.87425e-05   1.818    32  SYN
 1  entropy                2               7                    9  1          0             0.411     0  HTTP
 2  entropy                1              11                    2  0.496701   0             0.411     5  ACK
 3  gini                   4               1                    2  0.0506277  0.0201468     3.232    48  UDP
 4  gini                  32              31                    2  0.965399   0.00464241    0.225    49  ARP
 5  gini                   2              27                    7  1          0             0.375    16  SP
 6  gini                  20               7                    7  0.6642     0.130705      5.652    20  BF
 7  gini    

# LR

In [14]:
from scipy.stats import loguniform
from sklearn.linear_model import LogisticRegression
lines=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  

    df=pd.read_csv(file_list[j][1],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  

    X_train = X_train.dropna()
    y_train = y_train[X_train.index]
    X_test = X_test.dropna()
    y_test = y_test[X_test.index]
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
     'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty' : ['none', 'l1', 'l2', 'elasticnet'],
    'C' : loguniform(1e-5, 100)
        }
    
    second=time()
    f1=[]
    clf=LogisticRegression()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(LogisticRegression(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("LR_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    
final_parametres=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))

SYN
default                             0.9229715197382704   2.102    0       


100%|██████████| 10/10 [05:18<00:00, 31.88s/it]


HTTP
default                             0.3333333333333333   0.131    0       


100%|██████████| 10/10 [00:40<00:00,  4.04s/it]


ACK
default                             0.49670078103959064  0.142    0       


...

SCHD
default                             0.7975432319618366   0.259    0       


100%|██████████| 10/10 [00:49<00:00,  4.97s/it]


MHDis
default                             0.363789939490434    0.096    0       


100%|██████████| 10/10 [00:43<00:00,  4.39s/it]

               C  penalty    Solver              F1          Std    Time    No  Attack
--  ------------  ---------  ---------  -----------  -----------  ------  ----  --------
 0   0.285136     l2         newton-cg  0.95947      1.11022e-16  44.205     0  SYN
 1   0.710229     l2         newton-cg  0.95947      1.11022e-16  35.425     1  SYN
 2   0.00932418   l2         lbfgs      0.94789      0            11.246     2  SYN
 3   2.65609e-05  l2         newton-cg  0.940047     1.11022e-16  40.809     3  SYN
 4   0.213318     l2         newton-cg  0.95947      1.11022e-16  39.799     4  SYN
 5  39.7795       l2         newton-cg  0.964859     0            44.008     5  SYN
 6   0.00559524   l2         liblinear  0.837293     0             5.18      6  SYN
 7   0.034284     l2         newton-cg  0.95947      1.11022e-16  34.997     7  SYN
 8   7.00727e-05  l2         lbfgs      0.953145     0            22.909     8  SYN
 9   0.0125401    l2         newton-cg  0.946673     0            40